In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

file_path = '/kaggle/input/word-level/dict.csv'
file_path2='/kaggle/input/word-level/output_file.csv'
df1 = pd.read_csv(file_path)
df2=pd.read_csv(file_path2)

df1['Sanskrit'] = df1['Sanskrit'].apply(lambda x: x.lower().strip())
df1['English'] = df1['English'].apply(lambda x: x.lower().strip())

df2['Sanskrit'] = df2['Sanskrit'].apply(lambda x: x.lower().strip())
df2['English'] = df2['English'].apply(lambda x: x.lower().strip())

In [ ]:
df = pd.concat([df1[['Sanskrit', 'English']], df2[['Sanskrit', 'English']]], ignore_index=True)

In [ ]:
df

In [ ]:
# import pandas as pd
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# import numpy as np

# # Example DataFrame
# # df = pd.DataFrame({
# #     'Sanskrit': ['संस्कृत', 'धर्म', 'ज्ञान'],
# #     'English': ['sanskrit', 'dharma', 'knowledge']
# # })

# # Tokenize characters
# input_tokenizer = Tokenizer(char_level=True)
# target_tokenizer = Tokenizer(char_level=True)

# input_tokenizer.fit_on_texts(df['Sanskrit'])
# target_tokenizer.fit_on_texts(df['English'])

# input_sequences = input_tokenizer.texts_to_sequences(df['Sanskrit'])
# target_sequences = target_tokenizer.texts_to_sequences(df['English'])

# max_input_length = max(len(seq) for seq in input_sequences)
# max_target_length = max(len(seq) for seq in target_sequences)

# input_sequences = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
# target_sequences = pad_sequences(target_sequences, maxlen=max_target_length, padding='post')

# input_vocab_size = len(input_tokenizer.word_index) + 1
# target_vocab_size = len(target_tokenizer.word_index) + 1

In [ ]:
!pip install indic_transliteration

In [ ]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def transliterate_text(sanskrit_text):
    transliterated_text = transliterate(sanskrit_text, sanscript.DEVANAGARI, sanscript.IAST)
    return transliterated_text



In [ ]:
df['Transliterated_Sanskrit']=df['Sanskrit'].apply(transliterate_text)

In [ ]:
df

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv1D, LSTM, GRU, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

# Hyperparameters
embedding_dim = 128
filters = 64
kernel_size = 3
lstm_units = 256
dropout_rate = 0.5
max_seq_length = 100  
vocab_size_sanskrit = 256  
vocab_size_english = 256  

transliterated_sanskrit_texts = df['Transliterated_Sanskrit'].values
english_texts = df['English'].values

# Build character-level tokenizer for transliterated Sanskrit and English
def build_char_tokenizer(texts):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(texts)
    return tokenizer

transliterated_sanskrit_tokenizer = build_char_tokenizer(transliterated_sanskrit_texts)
english_tokenizer = build_char_tokenizer(english_texts)

# Convert texts to sequences
transliterated_sanskrit_sequences = transliterated_sanskrit_tokenizer.texts_to_sequences(transliterated_sanskrit_texts)
english_sequences = english_tokenizer.texts_to_sequences(english_texts)

# Pad sequences
transliterated_sanskrit_padded = pad_sequences(transliterated_sanskrit_sequences, maxlen=max_seq_length, padding='post')
english_padded = pad_sequences(english_sequences, maxlen=max_seq_length, padding='post')

# Build the model
input_layer = Input(shape=(max_seq_length,))  # Adjust input shape for character-level encoding

# Embedding layer
embedding_layer = Embedding(input_dim=vocab_size_sanskrit, output_dim=embedding_dim)(input_layer)

# CNN layer
cnn_layer = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(embedding_layer)

# LSTM or GRU layer (you can choose either)
lstm_layer = Bidirectional(LSTM(lstm_units, return_sequences=True))(cnn_layer)
# gru_layer = Bidirectional(GRU(lstm_units, return_sequences=True))(cnn_layer)

# Dense layer for output
output_layer = TimeDistributed(Dense(vocab_size_english, activation='softmax'))(lstm_layer)

# Create and compile the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
# Convert target sequences to 3D array of shape (batch_size, sequence_length, 1)
english_padded = np.expand_dims(english_padded, -1)
model.fit(transliterated_sanskrit_padded, english_padded, batch_size=64, epochs=20, validation_split=0.2)

# Save the model
# model.save('transliterated_sanskrit_to_english_translation_model.h5')

# Prediction
# Assuming `new_transliterated_sanskrit_texts` is a list of new transliterated Sanskrit sentences for prediction
# new_transliterated_sanskrit_sequences = transliterated_sanskrit_tokenizer.texts_to_sequences(new_transliterated_sanskrit_texts)
# new_transliterated_sanskrit_padded = pad_sequences(new_transliterated_sanskrit_sequences, maxlen=max_seq_length, padding='post')
# predictions = model.predict(new_transliterated_sanskrit_padded)

# # Decode predictions to English characters
# index_word_english = {v: k for k, v in english_tokenizer.word_index.items()}

# for prediction in predictions:
#     predicted_sequence = [index_word_english.get(i, '') for i in prediction.argmax(axis=1)]
#     print(''.join(predicted_sequence))


In [ ]:
model.save('/kaggle/working/hybrid.h5')
# model.fit([input_sequences, target_sequences], np.expand_dims(decoder_target_data, -1), batch_size=64, epochs=10, validation_split=0.2)

In [ ]:
# New transliterated Sanskrit sentence
new_sanskrit_sentence = "naama"

# Convert the new Sanskrit sentence to sequences using the tokenizer
new_transliterated_sanskrit_sequences = transliterated_sanskrit_tokenizer.texts_to_sequences([new_sanskrit_sentence])

# Pad the sequence to match the input length of the model
new_transliterated_sanskrit_padded = pad_sequences(new_transliterated_sanskrit_sequences, maxlen=max_seq_length, padding='post')

# Make predictions using the trained model
predictions = model.predict(new_transliterated_sanskrit_padded)

# Decode predictions to English characters
index_word_english = {v: k for k, v in english_tokenizer.word_index.items()}

# Convert predictions to text
predicted_sentence = ''.join([index_word_english.get(i, '') for i in predictions[0].argmax(axis=1)])

print("Predicted English translation:")
print(predicted_sentence)
